In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pylab
from qiskit import Aer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import NumPyEigensolver
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.initial_states import Zero
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.circuit.library import EfficientSU2
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit.test.mock import *

/Users/bavithra/miniconda3/envs/qiskit-env/lib/python3.8/site-packages/pyscf/lib/misc.py:47: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
plt.rcParams["figure.dpi"] = 100
plt.style.use("seaborn-pastel")

In [3]:
basis_sets = ['sto6g', '631g', 'ccpvdz']
threshold = 0.00000001
aqua_globals.random_seed = 500

In [4]:
def get_qubit_hamiltonian(d, basis_set, map_type='jordan_wigner'):
    """
    Obtain the qubit operators from the fermionic operators after second quantization.
    Args:
        d: inter-atomic distance
        basis_set: computational chemistry basis set
        map_type: encoding method

    Returns: qubit Hamiltonian as a weighted summed of Paulis, energy shift because of nuclear repulsion, number of particles and number of spin orbitals

    """
    driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(d) ,
                         unit=UnitsType.ANGSTROM,
                         charge=0,
                         spin=0,
                         basis=basis_set)
    molecule = driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    repulsion_energy = molecule.nuclear_repulsion_energy
    fermionic_op = FermionicOperator(h1,h2)
    qubit_op = fermionic_op.mapping(map_type=map_type, threshold=threshold)
    return qubit_op, repulsion_energy, num_particles, num_spin_orbitals

In [17]:
qubit_ops = {}
for set in basis_sets:
    qubit_ops["qubitOp_{0}".format(set)],_,_,_ = get_qubit_hamiltonian(0.735, set)

for label in qubit_ops:
    print("\n --- {} ---".format(label),qubit_ops[label])



 --- qubitOp_sto6g --- Representation: paulis, qubits: 4, size: 15

 --- qubitOp_631g --- Representation: paulis, qubits: 8, size: 185

 --- qubitOp_321g --- Representation: paulis, qubits: 8, size: 185


In [6]:
bond_lengths = np.linspace(0.1, 3.5, 5)

In [7]:
def construct_qinstance(device):
    simulator_backend = Aer.get_backend('qasm_simulator')

    device_backend = FakeValencia()
    qubit_connectivity = device_backend.configuration().coupling_map
    noise_model = NoiseModel.from_backend(device_backend)

    if device == "qasm_ideal":
        qinstance = QuantumInstance(backend=simulator_backend,
                                    seed_simulator=aqua_globals.random_seed)

    elif device == "fake_valencia":
        qinstance = QuantumInstance(backend=device_backend,
                                    noise_model=noise_model,
                                    coupling_map=qubit_connectivity,
                                    seed_simulator=aqua_globals.random_seed,
                                    seed_transpiler=aqua_globals.random_seed)

    elif device == "fake_valencia_with_mitigation":
        qinstance = QuantumInstance(backend=device_backend, seed_simulator=aqua_globals.random_seed,
                                    seed_transpiler=aqua_globals.random_seed,
                                    noise_model=noise_model,
                                    measurement_error_mitigation_cls=CompleteMeasFitter,
                                    cals_matrix_refresh_period=30)
    return qinstance

In [8]:
optimizers = [COBYLA, SPSA, SLSQP]

In [11]:
def store_intermediate_result_uccsd(eval_count, parameters, mean, std):
    counts_uccsd.append(eval_count)
    values_uccsd.append(mean)
    return counts_uccsd, values_uccsd

def store_intermediate_result_effSU2(eval_count, parameters, mean, std):
    counts_effSU2.append(eval_count)
    values_effSU2.append(mean)
    return counts_effSU2, values_effSU2

In [10]:
exact_energies = []

vqe_energies_effSU2 = []
vqe_energies_uccsd = []


counts_effSU2 = []
values_effSU2 = []

counts_uccsd = []
values_uccsd = []

In [ ]:
for dist in bond_lengths:
    counts = []
    values = []
    parameters = []
    qubitOp, shift, num_particles, num_spin_orbitals = get_qubit_hamiltonian(dist,basis_set='sto6g')
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(num_spin_orbitals,
                                num_particles,
                                two_qubit_reduction=False,
                                qubit_mapping='jordan_wigner')
    optimizer = SPSA(maxiter=200)

    ansatz = [UCCSD(num_orbitals= num_spin_orbitals,
                    num_particles= num_particles,
                    initial_state= initial_state,
                    qubit_mapping='jordan_wigner',
                    two_qubit_reduction=False,),
              EfficientSU2(num_qubits=qubitOp.num_qubits,
                           su2_gates=['rz','rx','rz'],
                           initial_state=Zero(qubitOp.num_qubits))]
    for psi in ansatz:
        if psi == ansatz[0]:
            vqe = VQE(qubitOp, psi, optimizer, callback=store_intermediate_result_uccsd)
            vqe_result = np.real(vqe.run(construct_qinstance('qasm_ideal'))['eigenvalue'] + shift)
            vqe_energies_uccsd.append(vqe_result)
        elif psi == ansatz[1]:
            vqe = VQE(qubitOp, psi, optimizer, callback=store_intermediate_result_effSU2)
            vqe_result = np.real(vqe.run(construct_qinstance('qasm_ideal'))['eigenvalue'] + shift)
            vqe_energies_effSU2.append(vqe_result)
        print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])

In [ ]:
pylab.plot(bond_lengths, exact_energies, label='NumPy Minimum Eigensolver')
pylab.plot(bond_lengths, vqe_energies_effSU2, label='VQE- Hardware efficient ansatz')
pylab.plot(bond_lengths, vqe_energies_uccsd, label='VQE- UCCSD')
pylab.xlabel('Interatomic distance')
pylab.ylabel('Energy')
pylab.title(r'$H_2$ Ground State Energy')
pylab.legend(loc='upper right');

In [ ]:
pylab.plot(distances, np.subtract(vqe_energies_effSU2, exact_energies), label='VQE- Hardware efficient ansatz')
pylab.plot(distances, np.subtract(vqe_energies_uccsd, exact_energies), label='VQE- UCCSD')
pylab.xlabel('Interatomic distance')
pylab.ylabel('Energy')
pylab.title('Energy difference from NumPyMinimumEigensolver')
pylab.legend(loc='upper left');